<h1><center>Fig 2 stats

This notebook contains statistical analysis of the data from the figure 2 and its supplemental figures.

# Install libraries

Install necessary libraries to run statistical tests in R.

In [ ]:
# Use R in Jupyter Notebook
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("readxl")
install.packages("tidyverse")
install.packages("emmeans")
install.packages("maptools", repos = "https://cloud.r-project.org")
install.packages("car", repos = "https://cloud.r-project.org")
install.packages("cowplot", repos = "https://cloud.r-project.org")
install.packages("rstatix", repos = "https://cloud.r-project.org")
install.packages("ggpubr", repos = "https://cloud.r-project.org")

In [ ]:
%%R
library("readxl")
library("tidyverse")
library("emmeans")
library("maptools")
library("car")
library("cowplot")
library("rstatix")
library("ggpubr")

# Directory

Set directory with the source data. Mount disk if you use Google Colab.

In [ ]:
# Mount disk if you use Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change directory
%cd /content/drive/MyDrive/!lab/Manuscripts/LHoutputs/full submission 2/sourcedata

/content/drive/MyDrive/!lab/Manuscripts/LHoutputs/full submission 2/sourcedata


# Total correlation Fig2A & SuppFig1A


In [ ]:
%%R
gcamp <- read_excel("fig2_LH_movementCorrelation.xlsx", sheet = "total")
gcamp["group"] = "GCaMP"
gcamp$group = as.factor(gcamp$group)
gcamp$test = as.factor(gcamp$test)
gcamp$output = as.factor(gcamp$output)
gcamp$R = as.numeric(gcamp$R)

In [ ]:
%%R
eyfp <- read_excel("fig2-supp1_LH-eYFP_movementCorrelation.xlsx", sheet = "total")
eyfp["group"] = "eYFP"
eyfp$group = as.factor(eyfp$group)
eyfp$test = as.factor(eyfp$test)
eyfp$output = as.factor(eyfp$output)
eyfp$R = as.numeric(eyfp$R)

In [ ]:
%%R
df <- rbind(gcamp,eyfp)

Outliers

In [ ]:
%%R
df %>%
  group_by(group,test,output) %>%
  identify_outliers(R)

# A tibble: 4 x 8
  test  output  group mouse     R    pvalue is.outlier is.extreme
  <fct> <fct>   <fct> <chr> <dbl>     <dbl> <lgl>      <lgl>     
1 OFT   LHA-LHb GCaMP LHA16 0.677 0         TRUE       TRUE      
2 OFT   LHA-VTA GCaMP LHA16 0.420 3.70e-251 TRUE       FALSE     
3 TST   LHA-LHb GCaMP LHA8  0.625 0         TRUE       FALSE     
4 TST   LHA-VTA GCaMP LHA16 0.332 1.10e-151 TRUE       FALSE     


Remove extreme outlier

In [ ]:
%%R
out <- (df$output=="LHA-LHb" & df$mouse=="LHA16" & df$test=="OFT")
df1 <- df[-which(out==TRUE), ]

Normality test

In [ ]:
%%R
gcamp %>%
  group_by(group,test,output) %>%
  shapiro_test(R)

# A tibble: 6 x 6
  test  output  group variable statistic      p
  <fct> <fct>   <fct> <chr>        <dbl>  <dbl>
1 OFT   LHA-DRN GCaMP R            0.958 0.748 
2 OFT   LHA-LHb GCaMP R            0.768 0.0431
3 OFT   LHA-VTA GCaMP R            0.981 0.965 
4 TST   LHA-DRN GCaMP R            0.956 0.774 
5 TST   LHA-LHb GCaMP R            0.798 0.0566
6 TST   LHA-VTA GCaMP R            0.922 0.521 


In [ ]:
%%R
eyfp %>%
  group_by(group,test) %>%
  shapiro_test(R)

# A tibble: 2 x 5
  test  group variable statistic     p
  <fct> <fct> <chr>        <dbl> <dbl>
1 OFT   eYFP  R            0.988 0.983
2 TST   eYFP  R            0.928 0.562


Find difference between groups

In [ ]:
%%R
res.aov <- anova_test(data=df1, dv=R, wid=mouse, between=group, within = c(test,output))
get_anova_table(res.aov)

ANOVA Table (type III tests)

             Effect DFn DFd      F     p p<.05   ges
1             group   1   2 82.264 0.012     * 0.819
2              test   1   2  9.817 0.089       0.200
3            output   2   4  1.208 0.389       0.208
4        group:test   1   2  9.845 0.088       0.200
5      group:output   2   4  0.271 0.776       0.056
6       test:output   2   4  0.393 0.699       0.074
7 group:test:output   2   4  1.859 0.269       0.274


In [ ]:
%%R
test.effect <- df1 %>%
  group_by(group, output) %>%
  anova_test(dv=R, wid=mouse, within=test) %>%
get_anova_table()
test.effect

# A tibble: 6 x 9
  output  group Effect   DFn   DFd      F     p `p<.05`   ges
* <fct>   <fct> <chr>  <dbl> <dbl>  <dbl> <dbl> <chr>   <dbl>
1 LHA-DRN GCaMP test       1     7  8.41  0.023 "*"     0.266
2 LHA-LHb GCaMP test       1     2 40.1   0.024 "*"     0.848
3 LHA-VTA GCaMP test       1     5  7.14  0.044 "*"     0.507
4 LHA-DRN eYFP  test       1     1  1.69  0.417 ""      0.374
5 LHA-LHb eYFP  test       1     1  0.661 0.565 ""      0.285
6 LHA-VTA eYFP  test       1     1  1.38  0.448 ""      0.221


In [ ]:
%%R
test.effect <- df1 %>%
  group_by(group,output) %>%
  tukey_hsd(R ~ test, paired=TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate)
test.effect

# A tibble: 6 x 8
  output  group group1 group2 conf.low conf.high   p.adj p.adj.signif
  <fct>   <fct> <chr>  <chr>     <dbl>     <dbl>   <dbl> <chr>       
1 LHA-DRN GCaMP OFT    TST    -0.00746     0.284 0.0615  ns          
2 LHA-LHb GCaMP OFT    TST     0.234       0.497 0.00021 ***         
3 LHA-VTA GCaMP OFT    TST     0.0410      0.184 0.00528 **          
4 LHA-DRN eYFP  OFT    TST    -0.595       1.00  0.389   ns          
5 LHA-LHb eYFP  OFT    TST    -0.625       0.410 0.466   ns          
6 LHA-VTA eYFP  OFT    TST    -0.641       0.450 0.53    ns          


Compare to 0

In [ ]:
%%R
df1 %>%
  group_by(group,test,output) %>%
  t_test(R ~ 0) %>%
  select(-statistic)

# A tibble: 12 x 9
   test  output  group .y.   group1 group2         n    df           p
   <fct> <fct>   <fct> <chr> <chr>  <chr>      <int> <dbl>       <dbl>
 1 OFT   LHA-DRN GCaMP R     1      null model    11    10 0.000909   
 2 OFT   LHA-LHb GCaMP R     1      null model     4     3 0.133      
 3 OFT   LHA-VTA GCaMP R     1      null model     7     6 0.00000156 
 4 TST   LHA-DRN GCaMP R     1      null model     8     7 0.000000713
 5 TST   LHA-LHb GCaMP R     1      null model     6     5 0.0000929  
 6 TST   LHA-VTA GCaMP R     1      null model     6     5 0.0000162  
 7 OFT   LHA-DRN eYFP  R     1      null model     2     1 0.178      
 8 OFT   LHA-LHb eYFP  R     1      null model     2     1 0.383      
 9 OFT   LHA-VTA eYFP  R     1      null model     2     1 0.978      
10 TST   LHA-DRN eYFP  R     1      null model     2     1 0.665      
11 TST   LHA-LHb eYFP  R     1      null model     2     1 0.484      
12 TST   LHA-VTA eYFP  R     1      null model     2     1

# Perievent correlation Fig2C & FigSupp1C

In [ ]:
%%R
gcamp <- read_excel("fig2_LH_movementCorrelation.xlsx", sheet="perievent")
gcamp["group"] = "GCaMP"
gcamp$group = as.factor(gcamp$group)
gcamp$test = as.factor(gcamp$test)
gcamp$output = as.factor(gcamp$output)
gcamp$event = as.factor(gcamp$event)
gcamp$posCorr = as.numeric(gcamp$posCorr)
gcamp$negCorr = as.numeric(gcamp$negCorr)
gcamp$notCorr = as.numeric(gcamp$notCorr)

In [ ]:
%%R
eyfp <- read_excel("fig2-supp1_LH-eYFP_movementCorrelation.xlsx", sheet="perievent")
eyfp["group"] = "eYFP"
eyfp$group = as.factor(eyfp$group)
eyfp$test = as.factor(eyfp$test)
eyfp$output = as.factor(eyfp$output)
eyfp$event = as.factor(eyfp$event)
eyfp$posCorr = as.numeric(eyfp$posCorr)
eyfp$negCorr = as.numeric(eyfp$negCorr)
eyfp$notCorr = as.numeric(eyfp$notCorr)

In [ ]:
%%R
df <- rbind(gcamp,eyfp)

In [ ]:
%%R
neworder <- c('immobility_off-on','immobility_offset','immobility_on-off','immobility_onset')
df <- arrange(transform(df, event=factor(event,levels=neworder)),event)
df$event <- factor(df$event, labels = c('mobile','mobOnset','immobile','immOnset'))

Outliers

In [ ]:
%%R
df %>%
  group_by(group,test,output,event) %>%
  identify_outliers(posCorr)

# A tibble: 9 x 10
  test  output  event  group mouse posCorr negCorr notCorr is.outlier is.extreme
  <fct> <fct>   <fct>  <fct> <chr>   <dbl>   <dbl>   <dbl> <lgl>      <lgl>     
1 OFT   LHA-DRN mobile GCaMP LHA10   0.04    0.1     0.86  TRUE       FALSE     
2 OFT   LHA-LHb mobile GCaMP LHA16   0.32    0.22    0.46  TRUE       FALSE     
3 OFT   LHA-LHb immob… GCaMP LHA16   0.56    0.1     0.34  TRUE       TRUE      
4 OFT   LHA-LHb immob… GCaMP LHA6    0.12    0.2     0.68  TRUE       FALSE     
5 OFT   LHA-VTA immOn… GCaMP LHA6    0.733   0.133   0.133 TRUE       FALSE     
6 TST   LHA-DRN mobile GCaMP LHA1    0.84    0.06    0.1   TRUE       FALSE     
7 TST   LHA-DRN immOn… GCaMP LHA9    0.25    0       0.75  TRUE       FALSE     
8 TST   LHA-LHb mobile GCaMP LHA16   0.36    0.2     0.44  TRUE       FALSE     
9 TST   LHA-LHb immob… GCaMP LHA14   0.02    0.16    0.82  TRUE       TRUE      


In [ ]:
%%R
df %>%
  group_by(group,test,output,event) %>%
  identify_outliers(negCorr)

# A tibble: 8 x 10
  test  output  event  group mouse posCorr negCorr notCorr is.outlier is.extreme
  <fct> <fct>   <fct>  <fct> <chr>   <dbl>   <dbl>   <dbl> <lgl>      <lgl>     
1 OFT   LHA-DRN immob… GCaMP LHA15   0.34   0.34    0.32   TRUE       FALSE     
2 OFT   LHA-LHb mobOn… GCaMP LHA16   0.852  0.0370  0.111  TRUE       FALSE     
3 OFT   LHA-LHb immOn… GCaMP LHA16   0.852  0       0.148  TRUE       FALSE     
4 OFT   LHA-VTA immob… GCaMP LHA14   0.28   0.02    0.7    TRUE       FALSE     
5 OFT   LHA-VTA immOn… GCaMP LHA15   0.533  0.267   0.2    TRUE       FALSE     
6 TST   LHA-LHb immob… GCaMP LHA15   0.32   0.14    0.54   TRUE       TRUE      
7 TST   LHA-LHb immob… GCaMP LHA16   0.3    0.32    0.38   TRUE       TRUE      
8 TST   LHA-VTA mobOn… GCaMP LHA15   0.947  0       0.0526 TRUE       FALSE     


Normality test

In [ ]:
%%R
shapiro <- gcamp %>%
  group_by(test,output,event) %>%
  shapiro_test(posCorr)
print(shapiro,n=24)

# A tibble: 24 x 6
   test  output  event             variable statistic       p
   <fct> <fct>   <fct>             <chr>        <dbl>   <dbl>
 1 OFT   LHA-DRN immobility_off-on posCorr      0.925 0.366  
 2 OFT   LHA-DRN immobility_offset posCorr      0.949 0.637  
 3 OFT   LHA-DRN immobility_on-off posCorr      0.972 0.902  
 4 OFT   LHA-DRN immobility_onset  posCorr      0.957 0.740  
 5 OFT   LHA-LHb immobility_off-on posCorr      0.974 0.902  
 6 OFT   LHA-LHb immobility_offset posCorr      0.838 0.159  
 7 OFT   LHA-LHb immobility_on-off posCorr      0.954 0.767  
 8 OFT   LHA-LHb immobility_onset  posCorr      0.993 0.989  
 9 OFT   LHA-VTA immobility_off-on posCorr      0.950 0.733  
10 OFT   LHA-VTA immobility_offset posCorr      0.901 0.334  
11 OFT   LHA-VTA immobility_on-off posCorr      0.964 0.853  
12 OFT   LHA-VTA immobility_onset  posCorr      0.886 0.252  
13 TST   LHA-DRN immobility_off-on posCorr      0.858 0.114  
14 TST   LHA-DRN immobility_offset posCorr      0.9

In [ ]:
%%R
shapiro <- gcamp %>%
  group_by(test,output,event) %>%
  shapiro_test(negCorr)
print(shapiro,n=24)

# A tibble: 24 x 6
   test  output  event             variable statistic        p
   <fct> <fct>   <fct>             <chr>        <dbl>    <dbl>
 1 OFT   LHA-DRN immobility_off-on negCorr      0.882 0.110   
 2 OFT   LHA-DRN immobility_offset negCorr      0.983 0.982   
 3 OFT   LHA-DRN immobility_on-off negCorr      0.858 0.0537  
 4 OFT   LHA-DRN immobility_onset  negCorr      0.945 0.576   
 5 OFT   LHA-LHb immobility_off-on negCorr      0.831 0.141   
 6 OFT   LHA-LHb immobility_offset negCorr      0.958 0.796   
 7 OFT   LHA-LHb immobility_on-off negCorr      0.814 0.105   
 8 OFT   LHA-LHb immobility_onset  negCorr      0.784 0.0598  
 9 OFT   LHA-VTA immobility_off-on negCorr      0.975 0.934   
10 OFT   LHA-VTA immobility_offset negCorr      0.912 0.409   
11 OFT   LHA-VTA immobility_on-off negCorr      0.880 0.226   
12 OFT   LHA-VTA immobility_onset  negCorr      0.869 0.181   
13 TST   LHA-DRN immobility_off-on negCorr      0.844 0.0832  
14 TST   LHA-DRN immobility_offset n

In [ ]:
%%R
eyfp %>%
  group_by(group,test,event) %>%
  shapiro_test(posCorr)

# A tibble: 8 x 6
  test  event             group variable statistic         p
  <fct> <fct>             <fct> <chr>        <dbl>     <dbl>
1 OFT   immobility_off-on eYFP  posCorr      0.941 0.668    
2 OFT   immobility_offset eYFP  posCorr      0.822 0.0911   
3 OFT   immobility_on-off eYFP  posCorr      0.960 0.820    
4 OFT   immobility_onset  eYFP  posCorr      0.838 0.125    
5 TST   immobility_off-on eYFP  posCorr      0.908 0.425    
6 TST   immobility_offset eYFP  posCorr      0.496 0.0000207
7 TST   immobility_on-off eYFP  posCorr      0.799 0.0578   
8 TST   immobility_onset  eYFP  posCorr      0.758 0.0241   


In [ ]:
%%R
eyfp %>%
  group_by(group,test,event) %>%
  shapiro_test(negCorr)

# A tibble: 8 x 6
  test  event             group variable statistic       p
  <fct> <fct>             <fct> <chr>        <dbl>   <dbl>
1 OFT   immobility_off-on eYFP  negCorr      0.855 0.174  
2 OFT   immobility_offset eYFP  negCorr      0.880 0.269  
3 OFT   immobility_on-off eYFP  negCorr      0.748 0.0191 
4 OFT   immobility_onset  eYFP  negCorr      0.908 0.421  
5 TST   immobility_off-on eYFP  negCorr      0.820 0.0884 
6 TST   immobility_offset eYFP  negCorr      0.762 0.0263 
7 TST   immobility_on-off eYFP  negCorr      0.666 0.00264
8 TST   immobility_onset  eYFP  negCorr      0.692 0.00514


MANOVA on positive and negative correlation

In [ ]:
%%R
man <- manova(cbind(posCorr, negCorr) ~ group*output*test*event, random=~1|mouse, data = df)
summary.manova(man)

                         Df  Pillai approx F num Df den Df    Pr(>F)    
group                     1 0.65122  159.640      2    171 < 2.2e-16 ***
output                    2 0.01931    0.838      4    344  0.501489    
test                      1 0.11848   11.492      2    171 2.076e-05 ***
event                     3 0.45248   16.764      6    344 < 2.2e-16 ***
group:output              2 0.01224    0.530      4    344  0.714084    
group:test                1 0.07153    6.587      2    171  0.001754 ** 
output:test               2 0.01989    0.864      4    344  0.485703    
group:event               3 0.25221    8.273      6    344 2.212e-08 ***
output:event              6 0.01095    0.158     12    344  0.999525    
test:event                3 0.18775    5.940      6    344 6.419e-06 ***
group:output:test         2 0.09454    4.267      4    344  0.002205 ** 
group:output:event        6 0.01829    0.265     12    344  0.993909    
group:test:event          3 0.08699    2.607      6

ANOVA post-hoc analysis

Negative correlation

In [ ]:
%%R 
anova <- aov(negCorr ~ group*output*test*event, random=~1|mouse, data=df)
summary(anova)

                         Df Sum Sq Mean Sq F value   Pr(>F)    
group                     1 0.0040 0.00396   0.369 0.544428    
output                    2 0.0138 0.00689   0.643 0.527018    
test                      1 0.0452 0.04517   4.213 0.041639 *  
event                     3 0.0760 0.02533   2.363 0.073005 .  
group:output              2 0.0180 0.00899   0.838 0.434266    
group:test                1 0.0079 0.00794   0.741 0.390615    
output:test               2 0.0151 0.00754   0.703 0.496312    
group:event               3 0.3333 0.11109  10.360 2.65e-06 ***
output:event              6 0.0050 0.00083   0.078 0.998179    
test:event                3 0.0574 0.01912   1.783 0.152161    
group:output:test         2 0.1794 0.08968   8.363 0.000342 ***
group:output:event        6 0.0303 0.00505   0.471 0.829122    
group:test:event          3 0.1111 0.03702   3.452 0.017848 *  
output:test:event         6 0.0212 0.00353   0.329 0.920806    
group:output:test:event   6 0.1262 0.021

Positive correlation

In [ ]:
%%R
anova <- aov(posCorr ~ group*output*test*event, random=~1|mouse, data = df)
summary(anova)

                         Df Sum Sq Mean Sq F value   Pr(>F)    
group                     1  6.323   6.323 285.586  < 2e-16 ***
output                    2  0.031   0.015   0.695 0.500591    
test                      1  0.502   0.502  22.695 4.02e-06 ***
event                     3  2.824   0.941  42.516  < 2e-16 ***
group:output              2  0.006   0.003   0.144 0.866140    
group:test                1  0.292   0.292  13.192 0.000371 ***
output:test               2  0.051   0.025   1.143 0.321386    
group:event               3  0.954   0.318  14.370 2.13e-08 ***
output:event              6  0.026   0.004   0.196 0.977601    
test:event                3  0.728   0.243  10.958 1.27e-06 ***
group:output:test         2  0.024   0.012   0.537 0.585515    
group:output:event        6  0.009   0.002   0.070 0.998620    
group:test:event          3  0.239   0.080   3.594 0.014839 *  
output:test:event         6  0.036   0.006   0.271 0.949683    
group:output:test:event   6  0.021   0.0

The main effect produced by difference in positive correlation.

#### Analysis for positive correlation

For positive correlation, factor output and its interaction with other factors does not have effect (p > 0.5). In further analysis, let's remove this factor.

Group effect

In [ ]:
%%R
group.effect <- df %>%
  group_by(test,event) %>%
  tukey_hsd(posCorr ~ group, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate)
group.effect

# A tibble: 8 x 8
  test  event    group1 group2 conf.low conf.high    p.adj p.adj.signif
  <fct> <fct>    <chr>  <chr>     <dbl>     <dbl>    <dbl> <chr>       
1 OFT   mobile   GCaMP  eYFP     -0.254   -0.0292 1.55e- 2 *           
2 OFT   mobOnset GCaMP  eYFP     -0.602   -0.280  5.98e- 6 ****        
3 OFT   immobile GCaMP  eYFP     -0.397   -0.158  5.53e- 5 ****        
4 OFT   immOnset GCaMP  eYFP     -0.623   -0.271  1.68e- 5 ****        
5 TST   mobile   GCaMP  eYFP     -0.535   -0.235  1.99e- 5 ****        
6 TST   mobOnset GCaMP  eYFP     -0.900   -0.760  2.1 e-14 ****        
7 TST   immobile GCaMP  eYFP     -0.323   -0.141  2.01e- 5 ****        
8 TST   immOnset GCaMP  eYFP     -0.708   -0.428  1.44e- 8 ****        


Test effect

In [ ]:
%%R
test.effect <- df %>%
  group_by(group,event) %>%
  tukey_hsd(posCorr ~ test, paired=TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate)
test.effect

# A tibble: 8 x 8
  event    group group1 group2 conf.low conf.high        p.adj p.adj.signif
  <fct>    <fct> <chr>  <chr>     <dbl>     <dbl>        <dbl> <chr>       
1 mobile   GCaMP OFT    TST      0.112    0.295   0.0000579    ****        
2 mobOnset GCaMP OFT    TST      0.242    0.424   0.0000000043 ****        
3 immobile GCaMP OFT    TST     -0.152    0.00109 0.0532       ns          
4 immOnset GCaMP OFT    TST     -0.0320   0.197   0.153        ns          
5 mobile   eYFP  OFT    TST     -0.147    0.0669  0.424        ns          
6 mobOnset eYFP  OFT    TST     -0.134    0.0216  0.138        ns          
7 immobile eYFP  OFT    TST     -0.0837   0.0237  0.241        ns          
8 immOnset eYFP  OFT    TST     -0.123    0.0462  0.335        ns          


Event effect

In [ ]:
%%R
event.effect <- df %>%
  group_by(group,test) %>%
  tukey_hsd(posCorr ~ event, paired=TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate)
event.effect %>% filter(group=="GCaMP")

# A tibble: 12 x 8
   test  group group1   group2   conf.low conf.high    p.adj p.adj.signif
   <fct> <fct> <chr>    <chr>       <dbl>     <dbl>    <dbl> <chr>       
 1 OFT   GCaMP mobile   mobOnset   0.124     0.381  8.89e- 6 ****        
 2 OFT   GCaMP mobile   immobile  -0.0456    0.211  3.37e- 1 ns          
 3 OFT   GCaMP mobile   immOnset   0.136     0.393  3.28e- 6 ****        
 4 OFT   GCaMP mobOnset immobile  -0.298    -0.0419 4.36e- 3 **          
 5 OFT   GCaMP mobOnset immOnset  -0.116     0.140  9.95e- 1 ns          
 6 OFT   GCaMP immobile immOnset   0.0537    0.310  1.98e- 3 **          
 7 TST   GCaMP mobile   mobOnset   0.270     0.495  0        ****        
 8 TST   GCaMP mobile   immobile  -0.309    -0.0833 1.08e- 4 ***         
 9 TST   GCaMP mobile   immOnset   0.0312    0.257  6.68e- 3 **          
10 TST   GCaMP mobOnset immobile  -0.691    -0.466  0        ****        
11 TST   GCaMP mobOnset immOnset  -0.351    -0.126  2.25e- 6 ****        
12 TST   GCaMP immo

In [ ]:
%%R
event.effect %>% filter(group=="eYFP")

# A tibble: 12 x 8
   test  group group1   group2   conf.low conf.high  p.adj p.adj.signif
   <fct> <fct> <chr>    <chr>       <dbl>     <dbl>  <dbl> <chr>       
 1 OFT   eYFP  mobile   mobOnset  -0.176    0.0822  0.743  ns          
 2 OFT   eYFP  mobile   immobile  -0.182    0.0755  0.659  ns          
 3 OFT   eYFP  mobile   immOnset  -0.170    0.0877  0.808  ns          
 4 OFT   eYFP  mobOnset immobile  -0.136    0.122   0.999  ns          
 5 OFT   eYFP  mobOnset immOnset  -0.123    0.134   0.999  ns          
 6 OFT   eYFP  immobile immOnset  -0.117    0.141   0.993  ns          
 7 TST   eYFP  mobile   mobOnset  -0.135    0.00874 0.0981 ns          
 8 TST   eYFP  mobile   immobile  -0.115    0.0283  0.353  ns          
 9 TST   eYFP  mobile   immOnset  -0.111    0.0320  0.428  ns          
10 TST   eYFP  mobOnset immobile  -0.0521   0.0912  0.869  ns          
11 TST   eYFP  mobOnset immOnset  -0.0484   0.0949  0.801  ns          
12 TST   eYFP  immobile immOnset  -0.0680   0

#### Analysis for negative correlation

Group effect

In [ ]:
%%R
group.effect <- df %>%
  group_by(test,output,event) %>%
  tukey_hsd(negCorr ~ group, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate)
print(group.effect,n=24)

# A tibble: 24 x 9
   test  output  event    group1 group2 conf.low conf.high   p.adj p.adj.signif
   <fct> <fct>   <fct>    <chr>  <chr>     <dbl>     <dbl>   <dbl> <chr>       
 1 OFT   LHA-DRN mobile   GCaMP  eYFP    -0.183    0.123   0.674   ns          
 2 OFT   LHA-DRN mobOnset GCaMP  eYFP     0.0141   0.319   0.035   *           
 3 OFT   LHA-DRN immobile GCaMP  eYFP    -0.255   -0.0157  0.03    *           
 4 OFT   LHA-DRN immOnset GCaMP  eYFP     0.0244   0.338   0.0273  *           
 5 OFT   LHA-LHb mobile   GCaMP  eYFP    -0.328   -0.0997  0.00483 **          
 6 OFT   LHA-LHb mobOnset GCaMP  eYFP    -0.417    0.0945  0.166   ns          
 7 OFT   LHA-LHb immobile GCaMP  eYFP    -0.199    0.135   0.643   ns          
 8 OFT   LHA-LHb immOnset GCaMP  eYFP    -0.341    0.167   0.42    ns          
 9 OFT   LHA-VTA mobile   GCaMP  eYFP    -0.327    0.0929  0.229   ns          
10 OFT   LHA-VTA mobOnset GCaMP  eYFP    -0.135    0.260   0.479   ns          
11 OFT   LHA-VTA immo

Test effect

In [ ]:
%%R
test.effect <- df %>%
  group_by(group,output,event) %>%
  tukey_hsd(negCorr ~ test, paired=TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate)
print(test.effect,n=24)

# A tibble: 24 x 9
   output  event    group group1 group2 conf.low conf.high    p.adj p.adj.signif
   <fct>   <fct>    <fct> <chr>  <chr>     <dbl>     <dbl>    <dbl> <chr>       
 1 LHA-DRN mobile   GCaMP OFT    TST     -0.0724   0.102   7.22e- 1 ns          
 2 LHA-DRN mobOnset GCaMP OFT    TST     -0.0995   0.0197  1.76e- 1 ns          
 3 LHA-DRN immobile GCaMP OFT    TST     -0.0379   0.117   2.96e- 1 ns          
 4 LHA-DRN immOnset GCaMP OFT    TST     -0.129    0.0148  1.12e- 1 ns          
 5 LHA-LHb mobile   GCaMP OFT    TST     -0.216   -0.0390  9.84e- 3 **          
 6 LHA-LHb mobOnset GCaMP OFT    TST     -0.319   -0.0645  7.79e- 3 **          
 7 LHA-LHb immobile GCaMP OFT    TST     -0.0514   0.114   4.14e- 1 ns          
 8 LHA-LHb immOnset GCaMP OFT    TST     -0.212    0.0227  1.01e- 1 ns          
 9 LHA-VTA mobile   GCaMP OFT    TST     -0.162    0.107   6.66e- 1 ns          
10 LHA-VTA mobOnset GCaMP OFT    TST     -0.106    0.0108  1   e- 1 ns          
11 LHA-VT

Output effect

In [ ]:
%%R
output.effect <- df %>%
  group_by(group,test,event) %>%
  tukey_hsd(negCorr ~ output, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate)
print(output.effect,n=48)

# A tibble: 48 x 9
   test  event    group group1  group2  conf.low conf.high  p.adj p.adj.signif
   <fct> <fct>    <fct> <chr>   <chr>      <dbl>     <dbl>  <dbl> <chr>       
 1 OFT   mobile   GCaMP LHA-DRN LHA-LHb  -0.0442    0.212  0.246  ns          
 2 OFT   mobile   GCaMP LHA-DRN LHA-VTA  -0.118     0.112  0.998  ns          
 3 OFT   mobile   GCaMP LHA-LHb LHA-VTA  -0.226     0.0524 0.278  ns          
 4 OFT   mobOnset GCaMP LHA-DRN LHA-LHb   0.0127    0.209  0.0253 *           
 5 OFT   mobOnset GCaMP LHA-DRN LHA-VTA  -0.101     0.0756 0.931  ns          
 6 OFT   mobOnset GCaMP LHA-LHb LHA-VTA  -0.231    -0.0169 0.0216 *           
 7 OFT   immobile GCaMP LHA-DRN LHA-LHb  -0.0920    0.0851 0.995  ns          
 8 OFT   immobile GCaMP LHA-DRN LHA-VTA  -0.106     0.0525 0.673  ns          
 9 OFT   immobile GCaMP LHA-LHb LHA-VTA  -0.120     0.0727 0.813  ns          
10 OFT   immOnset GCaMP LHA-DRN LHA-LHb  -0.0517    0.187  0.342  ns          
11 OFT   immOnset GCaMP LHA-DRN L

Event effect

In [ ]:
%%R
event.effect <- df %>%
  group_by(group,output,test) %>%
  tukey_hsd(negCorr ~ event, paired=TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate)
print(event.effect,n=72)

# A tibble: 72 x 9
   test  output  group group1   group2   conf.low conf.high   p.adj p.adj.signif
   <fct> <fct>   <fct> <chr>    <chr>       <dbl>     <dbl>   <dbl> <chr>       
 1 OFT   LHA-DRN GCaMP mobile   mobOnset  -0.150   0.0238   0.226   ns          
 2 OFT   LHA-DRN GCaMP mobile   immobile  -0.111   0.0624   0.873   ns          
 3 OFT   LHA-DRN GCaMP mobile   immOnset  -0.148   0.0258   0.251   ns          
 4 OFT   LHA-DRN GCaMP mobOnset immobile  -0.0484  0.126    0.637   ns          
 5 OFT   LHA-DRN GCaMP mobOnset immOnset  -0.0849  0.0890   1       ns          
 6 OFT   LHA-DRN GCaMP immobile immOnset  -0.123   0.0504   0.675   ns          
 7 TST   LHA-DRN GCaMP mobile   mobOnset  -0.222  -0.0137   0.022   *           
 8 TST   LHA-DRN GCaMP mobile   immobile  -0.104   0.104    1       ns          
 9 TST   LHA-DRN GCaMP mobile   immOnset  -0.237  -0.0287   0.00846 **          
10 TST   LHA-DRN GCaMP mobOnset immobile   0.0137  0.222    0.022   *           
11 TST   

# Cross correlation Fig2D

In [ ]:
%%R
df <- read_excel("fig2_LH_movementCorrelation.xlsx", sheet="crossCorrelation")
df$test = as.factor(df$test)
df$output = as.factor(df$output)
df$maxlag = as.numeric(df$maxlag)

Outliers

In [ ]:
%%R
df %>%
  group_by(test,output) %>%
  identify_outliers(maxlag)

# A tibble: 3 x 6
  output  test  mouse maxlag is.outlier is.extreme
  <fct>   <fct> <chr>  <dbl> <lgl>      <lgl>     
1 LHA-DRN OFT   LHA6    -0.4 TRUE       FALSE     
2 LHA-LHb OFT   LHA6    -1.1 TRUE       FALSE     
3 LHA-LHb TST   LHA15   -0.3 TRUE       FALSE     


No extreme outliers.

Normality test

In [ ]:
%%R
df %>%
  group_by(test,output) %>%
  shapiro_test(maxlag)

# A tibble: 6 x 5
  output  test  variable statistic       p
  <fct>   <fct> <chr>        <dbl>   <dbl>
1 LHA-DRN OFT   maxlag       0.742 0.00165
2 LHA-LHb OFT   maxlag       0.956 0.782  
3 LHA-VTA OFT   maxlag       0.835 0.0891 
4 LHA-DRN TST   maxlag       0.957 0.778  
5 LHA-LHb TST   maxlag       0.947 0.719  
6 LHA-VTA TST   maxlag       0.813 0.0765 


Find difference between groups

In [ ]:
%%R
anova <- anova_test(data=df, dv=maxlag, wid=mouse, within=c(output,test))
get_anova_table(anova)

ANOVA Table (type III tests)

       Effect DFn DFd      F     p p<.05   ges
1      output   2   4  0.836 0.497       0.075
2        test   1   2 11.808 0.075       0.641
3 output:test   2   4  3.880 0.116       0.114


In [ ]:
%%R
test.effect <- df %>%
  group_by(output) %>%
  tukey_hsd(maxlag ~ test, paired=TRUE, p.adjust.method="bonferroni") %>%
  select(-term,-null.value,-estimate)
test.effect

# A tibble: 3 x 7
  output  group1 group2 conf.low conf.high    p.adj p.adj.signif
  <fct>   <chr>  <chr>     <dbl>     <dbl>    <dbl> <chr>       
1 LHA-DRN OFT    TST      -0.720   -0.266  0.000264 ***         
2 LHA-LHb OFT    TST      -1.27     0.0302 0.0594   ns          
3 LHA-VTA OFT    TST      -0.976   -0.162  0.0106   *           


In [ ]:
%%R
output.effect <- df %>%
  group_by(test) %>%
  tukey_hsd(maxlag ~ output, paired=TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate)
output.effect

# A tibble: 6 x 7
  test  group1  group2  conf.low conf.high p.adj p.adj.signif
  <fct> <chr>   <chr>      <dbl>     <dbl> <dbl> <chr>       
1 OFT   LHA-DRN LHA-LHb   -0.348     0.552 0.836 ns          
2 OFT   LHA-DRN LHA-VTA   -0.236     0.571 0.554 ns          
3 OFT   LHA-LHb LHA-VTA   -0.423     0.554 0.938 ns          
4 TST   LHA-DRN LHA-LHb   -0.498     0.448 0.99  ns          
5 TST   LHA-DRN LHA-VTA   -0.382     0.565 0.874 ns          
6 TST   LHA-LHb LHA-VTA   -0.389     0.623 0.826 ns          


Compare to 0

In [ ]:
%%R
df %>%
  group_by(test,output) %>%
  t_test(maxlag ~ 0) %>%
  select(-statistic)

# A tibble: 6 x 8
  output  test  .y.    group1 group2         n    df         p
  <fct>   <fct> <chr>  <chr>  <chr>      <int> <dbl>     <dbl>
1 LHA-DRN OFT   maxlag 1      null model    11    10 0.0000419
2 LHA-LHb OFT   maxlag 1      null model     5     4 0.8      
3 LHA-VTA OFT   maxlag 1      null model     7     6 0.881    
4 LHA-DRN TST   maxlag 1      null model     8     7 0.00088  
5 LHA-LHb TST   maxlag 1      null model     6     5 0.000782 
6 LHA-VTA TST   maxlag 1      null model     6     5 0.0187   
